## Introduction
#### The library that you will use for developing models will depend on the application.
#### Usually many statistical problems can be solved with simpler techniques. eg - ordinary least squares.
#### But other problems may require more advanced machine learning methods.
#### We will focus on the features that will be helpful while crossing back and forth in data wrangling and model fitting and scoring.

## Interfacing between pandas and Model Code
#### A common workflow for model development is to use pandas for data loading and cleaning, then switching to modeling library to build the model.
#### An important step in model development is 'feature engineering' i.e. data transformation or analytics that extracts information from raw dataset that will be useful in modeling context.
#### We often use the data aggregation and GroupBy tools in this step.
#### The point of contact between pandas and other analysis libraries is usually 'NumPy arrays'.
#### To turn DataFrame into NumPy array, use the '.values' property. To convert back to a DataFrame, you can pass a 2-d ndarray with optional column names to 'DataFrame'.
#### NOTE - '.values' is intended for homogenous data (eg - all numeric type). For heterogenous data, the result would be an ndarray of Python 'objects'. 

In [3]:
import pandas as pd
import numpy as np

data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})

data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [5]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [7]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [8]:
# Heterogenous data
df3 = data.copy()
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [9]:
# ndarray of object data
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

#### Some libraries have naive support for pandas to do this work automatically i.e. converting to NumPy and attaching model parameter names.
#### In some other scenarios, you may have to perform this 'metadata management' manually.
#### One example is to replace 'category' column with 'dummy' variables. And then dropping the category column and join the dummay variables to the main dataset.

In [11]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                 categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [12]:
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## Creating Model Descriptions with Patsy
#### There are some nuances to using statistical models with datasets having dummy variables.
#### A simpler and less error-prone way is to use 'Patsy', a Python library fro describing models (esp. linear models), using a small string-based 'formula syntax'.
#### It is well supported for linear models in 'statsmodels'. It's 'formulas' are special string syntax. (eg - y ~ x0 + x1).
#### The above syntax does not mean 'add x0 to x1', rather these are 'terms' in the 'design matrix' created for the model.
#### The 'patsy.dmatrices' takes a fromula string with a dataset (DataFrame or dict of arrays) and produces design matrices for linear model.

In [14]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})

data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [15]:
import patsy

y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [16]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [17]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

#### The DesignMatix instances are NumPy ndarrays with additional metadata.
#### The 'Intercept' is a convention for linear models (eg - OLS {Ordinary Least Squares}). By default it is 1. You can suppress it by adding the term '+ 0' to the model.
#### Patsy objects can be directly be directly passed into algorithms (eg - numpy.linalg.lstsq for OLS) which will perform the regression.
#### The model metadata is retained in 'design_info' attribute which we can reattach the model column names to the fitted coefficients to get a Series.

In [19]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [20]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [21]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

In [22]:
coef, resid, _, _ = np.linalg.lstsq(X, y)

C:\Users\adity\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [23]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [24]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Data Transformation in Patsy Formulas
#### We can mix Python code into Patsy formulas. While evaluating the formula, the library will find the functions you use in enclosing scope.
#### Patsy has built-in functions for commonly used variable transformations (eg - standardizing {nean 0 and variance 1}, centering {subtracting by mean}).

In [30]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})

y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [31]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

#### A part of the modeling process is to fit model on a dataset and then evaluate it using another datset (a 'hold-out' portion or nes data observed later).
#### When applying transformations (eg - standardize and center), you should be careful using the model for predictions.
#### These are called 'stateful' transformations, as you may use statistics (eg - mean, standard deviation) of original dataset when transforming new dataset.
#### The function 'patsy.build_design_matrices' can apply transformations to 'out-of-sample' data using saved information from original 'in-sample' dataset.

In [33]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]
})

new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

#### The plus (+) symbol does not mean addition in context of Patsy formulas. To add columns from dataset, you must wrap them in special 'l' function.
#### Patsy has several other built-in transforms in its 'patsy.builtins' module, which can be explored through its documentation.

In [35]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### Categorical Data and Patsy
#### Non-numeric data can be transformed for model design matrix in different ways.
#### When we use non-numeric terms in Patsy formula, they are converted to dummy variables by default.
#### If you have included Intercept, one of the levels will be left out to avoid collinearity.
#### If you omit the Intercept, columns for each category will be included in the design matrix. NOTE - Now it just shows a single column.

In [41]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

y, X = patsy.dmatrices('v2 ~ key1', data)

X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [42]:
# Omitting Intercept just shows a single column now.
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

#### To convert a Numeric columns into categorica, use the 'C' function.
#### Things can get complicated when using multiple categorical terms. This can happen because of using interaction terms of the form 'key1:key2'. (which can be used in ANOVA {Analysis of Variance} models).
#### Patsy provides other ways to transform categorical data (eg - transformations for terms with ordering), which can be seen in it's online documentation.

In [44]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [45]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [46]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## Introduction to statsmodel
#### statsmodel is a Python library fro fitting many kinds of statistical models, performing statistical tests and data exploration and visualization.
#### It has more 'classical' frequentist statistical methods. The Bayesian and machine learning models are in other libraries.
#### The kinds of models in statsmodel are:
####     1. Linear models, generalized linear models and robust linear models
####     2. Linear mixed effects models
####     3. Analysis of variance (ANOVA) methods
####     4. Time series processes and state space models
####     5. Generalized methods of moments

### Estimating Linear Models
#### There are several kinds of linear regression models in statsmodels, from basic (eg - ordinary least squares) to more complex (eg - iteratively reweighted least squares).
#### Linear models have 2 different main interfaces - array-based and formula-based. Both are accessed through API module imports.
#### As an example, we will create normally distributed data. The helper function 'dnorm' generates normally distributed data with given mean and variance.
#### We will then generate a linear model from the data with known parameters 'beta'.

In [50]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
        return mean + np.sqrt(variance) * np.random.randn(*size)
    
np.random.seed(12345)
N = 100
X = np.c_[dnorm(0, 0.4, size=N),
         dnorm(0, 0.6, size=N),
         dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [51]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [52]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

#### To add an intercept column to the existing matrix, we can use 'sm.add_constant' function.
#### To use ordinary least squares linear regression, we will use 'sm.OLS'. The 'fit' method will return regression results object containing estimated model parameters and other diagnostics.
#### The 'summary' method on the fit output 'results' can give a detailing diagnostic output of the model.

In [54]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [55]:
model = sm.OLS(y, X)

results = model.fit()

results.params

array([0.17826108, 0.22303962, 0.50095093])

In [56]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.430
Model:                            OLS   Adj. R-squared:                  0.413
Method:                 Least Squares   F-statistic:                     24.42
Date:                Sat, 01 Sep 2018   Prob (F-statistic):           7.44e-12
Time:                        17:49:02   Log-Likelihood:                -34.305
No. Observations:                 100   AIC:                             74.61
Df Residuals:                      97   BIC:                             82.42
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.1783      0.053      3.364      0.0

#### In above example, the parameters had generic names (x1, x2). Instead the parmeter names might be in a DataFrame.
#### In such a scenario, we can use statsmodels formula API and Patsy formula strings.
#### The result returned by statsmodels is a Series with the DataFrame column names attached. Also, we do not need to use 'add_constant' using formulas and pandas objects.
#### For a new out-of-sample data, you can compute predicted values with the estimated model parameters.
#### There are additional tools for analysis, diagnostics and visualizations and other kinds of linear models in statsmodels which can be explored in its documentation.

In [58]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [59]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [60]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

In [61]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

### Estimating Time Series Process
#### statsmodels also has a class of models for time series analysis. These include autogressive process, Kalman filtering and other space models, and multivariate autogressive models.
#### As an example we will simulate a time series with autoregressive structure and noise with AR(2) structure (i.e. 2 lags) of parameters 0.8 and -0.4.
#### You may not know the exact number of lagged terms when fitting AR model. So we can fit with some larger number of lags.
#### The estimated parameters in results have intercepts first and estimates for the first 2 lags next.

In [63]:
init_x = 4

import random
values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)

for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [64]:
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)

In [65]:
results.params

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])

## Introduction to scikit-learn
#### scikit-learn is one of the most widely used and trusted Python machine learning toolkits.
#### It contains a broad selection of standard supervised and unsupervised ML methods with tools for model selection and evaluation, data transformation, data loading, and model persistence.
#### These models can be used for classification, clustering, prediction and other common tasks.
#### An example would be the passenger survival rates on the Titanic which is present in Kaggle. The first step is downloading test and training dataset using pandas.
#### Libraries like statsmodel and scikit-learn cannot be fed missing data, so we need to look at columns to see if they contain missing data.

In [67]:
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')

train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [68]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [70]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

#### In statistics and ML, a typical task is to make predictions based on data features. eg - predict if a passenger would survive.
#### A model is fitted on training dataset and evaluated on an out-of-sample testing dataset.
#### Age can be used as a predictor here, but it has missing data. There are a number of ways to imputing missing data. But a simple one is fill the nulls with median.
#### Next we will encode the 'Sex' column as categorical column 'IsFemale'.

In [75]:
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

In [76]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

#### We next decide on the variables to include in our input and output for the ML algorithm and create NumPy arrays.
#### Now these steps are not enough for a 'good' ML model without proper feature engineering.
#### We use the 'LogisticRegression' model from scikit-learn and create model instance.

In [78]:
predictors = ['Pclass', 'IsFemale', 'Age']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values

X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [79]:
y_train[:5]

array([0, 1, 1, 1, 0], dtype=int64)

In [80]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

#### Just like statsmodels, we can fit the model instance to the training data using the 'fit' method.
#### We can then form predictions for test dataset using 'model.predict'. Using the actual values in the test dataset, you can compute accuracy percentage or some other 'error metric'.

In [82]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [83]:
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

In [87]:
# No actual values provided in test dataset, so using 'y_true' as proxy
'''
(y_true == y_predict).mean()
'''

'\n(y_true == y_predict).mean()\n'

#### Actually, there are many layers of complexity in model training. 
#### Many models have parameters that need to be tuned and there are techniques called 'cross-validation' used for parameter tuning to avoid overfitting on training data.
#### This yields a better predictive performance or robustness on new data.

#### Cross-validation works by splitting training data to simulate out-of-sample prediction.
#### Then, by using model accuracy score (eg - mean squared error), we can perform a grid search on model parameters i.e. predictors and predicted variables.
#### Some models (eg - Logistic regression) have estimator class with built-ib cross-validation.
#### eg - the 'LogisticRegressionCV' class can be used with a aprameter indicating how fine-grained the grid-search has to be.

In [90]:
from sklearn.linear_model import LogisticRegressionCV

model_cv = LogisticRegressionCV(10)

model_cv.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0)

#### To do the cross-validation manually, you can use the 'cross_val_score' helper function, which handles the data splitting process.
#### The default scoring metric for cross-validation is model-dependant, but you can choose an explicit scoring function.
#### Cross-validated models take longer to train, but often yield better model performance.

In [92]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=10)

scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([0.77232143, 0.80269058, 0.77027027, 0.78828829])